In [1]:
# import required libraries
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn

from restructure import *
from func import *

### limitations/ edge cases of the flooding rule:

-  if the model[-1].bias > y_ref, then y_ref never will be reached.
-  a large stepwith can result in poor a_refs.
-  top_layer normalization (rescale-top) was not introduced in [1] but we found that it can be helpful to generate more faithful explanations.
-  the flooding rule does not ensure that the offset found lays within the distribution of activations, especially when y_ref>>y.
-  an asymetric treatment of activations based on their respective weights in the last layer can be reasonable (for example in section VI. A).

In [2]:
model_path = '../currentModel/model.pt'

# Load
device = torch.device('cpu')
model = get_model()
model.load_state_dict(torch.load(model_path, map_location=device))

In [ ]:
data_dir='../Training_Data/'
batch_size=64
gene="RUBCNL"
criterion=nn.MSELoss()
learning_rate= 0.00001
optimizer = optim.AdamW([{"params": model.pretrained.parameters(), "lr": learning_rate},
                        {"params": model.my_new_layers.parameters(), "lr": learning_rate}], weight_decay=0.005)

In [4]:
train_loader, val_loader = get_data_loaders(data_dir, batch_size, gene)

In [ ]:
def train_epoch_tmp(resnet, device, dataloader, criterion, optimizer):
    train_loss = 0.0
    batch_corr_train = 0.0
    resnet.train()
    i = 0
    for images, labels, path in dataloader:
        if i % 20 == 0:
            print("train_epoch iteration ", i)
        i+=1
        images = images.to(device)
        images = images.float()

        labels = torch.stack(labels, dim=1)
        labels = labels.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        g_output = resnet(images)
        loss = criterion(g_output, labels)
        if i % 1000 == 0:
            print("lost")
        loss.backward()
        if i % 1000 == 0:
            print("backward")
        optimizer.step()
        train_loss += loss.item()

        corr = stats.pearsonr(g_output[:, 0].cpu().detach().numpy(), labels[:, 0].cpu().detach().numpy())[0]
        batch_corr_train += corr
        once = False
        return

    return train_loss, batch_corr_train
train_epoch_tmp(model, device, train_loader, criterion, optimizer)

In [14]:
model_restructured = restructure_model(model.gene1, 0, in_layer=-3, out_layer=-1)

In [15]:
print(model_restructured)
print(model.gene1)

In [11]:
model.gene1 = model_restructured

In [4]:
# find a_ref

i_sample = 60
y_ref = 1

input_sample = input_[i_sample].reshape(1,-1)
output_sample = model(input_sample).detach().numpy()[0][0]

plot_setting(model, input_, i_sample, output_sample, y_ref)

a_ref = find_a_ref(model, input_sample, y_ref) 

print(f'a i_sample:     {model[:-1](input_sample)}')
print(f'a_ref i_sample: {a_ref}')

print(f'y_ with a_ref:  {model[-1:](a_ref)}')

In [ ]:
# restructure model

model_restructured = restructure_model(model, a_ref, in_layer=-3, out_layer=-1)

print(model(input_sample))
print(model_restructured(input_sample))

##### The output of the model is reduced by y_ref for sample i. Therefore, explaining the model_restructured for input_sample using the respective LRP rules yields a contextualized explanation relative to y_ref.

Reference: 

[1] S. Letzgus, P. Wagner, J. Lederer, W. Samek, K.-R. M ̈uller, and G. Montavon. Toward explainable artificial intelligence for regression models: A methodological perspective. IEEE Signal Processing Magazine, 39(4):40–58, 2022.